In [10]:
import os
import zipfile
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from tqdm import tqdm


In [9]:
# Extract zip files
def extract_zip(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

extract_zip("train.zip", "dataset/train")
extract_zip("test1.zip", "dataset/test1")

In [13]:
class CatDogDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.files = [f for f in os.listdir(directory) if f.endswith(".jpg")]
        self.directory = directory
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img_name = self.files[idx]
        label = 0 if 'cat' in img_name.lower() else 1
        img_path = os.path.join(self.directory, img_name)
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

In [14]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

In [15]:
train_dir = "dataset/train/train"
full_dataset = CatDogDataset(train_dir, transform=transform)

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # 2 classes: cat and dog
model = model.to(device)

c:\Users\fiar fly\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\fiar fly\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\fiar fly/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100.0%


In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [19]:
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = 100 * correct / total
    print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}, Accuracy: {train_acc:.2f}%")

Epoch 1, Loss: 36.8085, Accuracy: 95.23%
Epoch 2, Loss: 7.4370, Accuracy: 99.20%
Epoch 3, Loss: 4.6372, Accuracy: 99.52%
Epoch 4, Loss: 3.3157, Accuracy: 99.64%
Epoch 5, Loss: 2.9300, Accuracy: 99.64%


In [21]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy: {100 * correct / total:.2f}%")

Validation Accuracy: 97.02%


In [22]:
test_dir = "dataset/test1/test1"
test_dataset = CatDogDataset(test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()
print("\n🧪 Sample Predictions on Test Data:")
with torch.no_grad():
    for i, (images, _) in enumerate(test_loader):
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        for j in range(len(preds)):
            print(f"Test Image {i * 32 + j}: {'Dog' if preds[j] == 1 else 'Cat'}")
        break  # Only print predictions for first batch


🧪 Sample Predictions on Test Data:
Test Image 0: Dog
Test Image 1: Cat
Test Image 2: Cat
Test Image 3: Dog
Test Image 4: Dog
Test Image 5: Cat
Test Image 6: Cat
Test Image 7: Dog
Test Image 8: Cat
Test Image 9: Cat
Test Image 10: Cat
Test Image 11: Cat
Test Image 12: Cat
Test Image 13: Cat
Test Image 14: Cat
Test Image 15: Dog
Test Image 16: Dog
Test Image 17: Cat
Test Image 18: Cat
Test Image 19: Cat
Test Image 20: Dog
Test Image 21: Cat
Test Image 22: Cat
Test Image 23: Cat
Test Image 24: Cat
Test Image 25: Dog
Test Image 26: Cat
Test Image 27: Cat
Test Image 28: Dog
Test Image 29: Dog
Test Image 30: Dog
Test Image 31: Dog


In [23]:
# Save the model after training
torch.save(model.state_dict(), "best_model.pth")
